# Neural Network

## Implementasi Oleh Kelompok

In [60]:
import math
import numpy as np

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [84]:
class NeuralNet:
    
    # Constructor
    def __init__(self, nodes_n_in_hidden_layers, learning_rate, momentum):
        nodes_n_in_hidden_layers.append(1) # satu node buat output
        self.nodes_n_in_hidden_layers = nodes_n_in_hidden_layers
        self.inputs = []
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.outputs = [] # output dari setiap node pada satu iterasi
        self.weights = [] # weight dari setiap edge pada satu iterasi
        self.biases = [] # bias dari setiap node pada satu iterasi
        self.weight_biases = []
        self.local_gradients = [] # local gradient dari setiap node pada satu iterasi
        self.delta_weights = [] # delta weight dari setiap edge pada satu iterasi
        self.delta_biases = [] # delta bias dari setiap node pada satu iterasi
        self.layer_nodes = [] # node-node pada layer-layer
        self.v = [] # v pada setiap node
        self.targets = 0
    
    # Feed Forward
    def feed_forward(self, datum_idx):
        for i in range (1, len(self.layer_nodes)):
            for j in range (0, len(self.layer_nodes[i])):
                current_node = self.layer_nodes[i][j]
                weights = []
                weights.append(self.biases[current_node])
                inputs = []
                inputs.append(1)
                for k in range(0, len(self.layer_nodes[i-1])):
                    if (self.weights[self.layer_nodes[i-1][k]][current_node] != None):
                            weights.append(self.weights[self.layer_nodes[i-1][k]][current_node])
                            
                    if (i==1):
                        inputs.append(self.inputs[datum_idx][self.layer_nodes[i-1][k]])
                    else:
                        inputs.append(self.outputs[self.layer_nodes[i-1][k]])
                v = np.dot(inputs, weights)
                self.v[current_node] = v
                self.outputs[current_node] = sigmoid(v)
                
    # Back Propagation
    def back_propagation(self):
        for i in range(len(self.layer_nodes)-1, 0, -1):
            for j in range(len(self.layer_nodes[i])-1, -1, -1):
                current_node = self.layer_nodes[i][j]
                if (i == len(self.layer_nodes)-1):
                    v = self.v[current_node]
                    sig_v = sigmoid(v)
                    self.local_gradients[current_node] = self.local_gradients[current_node] + (sig_v * (1 - sig_v) * (self.target - self.outputs[current_node]))
                else:
                    v = self.v[current_node]
                    sig_v = sigmoid(v)
                    weight_delta = 1
                    for k in range(0, len(self.layer_nodes[i+1])):
                        if (self.weights[current_node][self.layer_nodes[i+1][k]] != None):
                            weight_delta = weight_delta * self.local_gradients[self.layer_nodes[i+1][k]] * self.weights[current_node][self.layer_nodes[i+1][k]]
                    self.local_gradients[current_node] = self.local_gradients[current_node] + (sig_v * (1 - sig_v) * weight_delta)            
                
    # Update Weight
    def update_weight(self):
        for i in range(0, len(self.layer_nodes)-1):
            for j in range(0, len(self.layer_nodes[i])):
                current_node = self.layer_nodes[i][j]
                for k in range(0, len(self.layer_nodes[i+1])):
                    current_next_node = self.layer_nodes[i+1][k]
                    new_weight = self.weights[current_node][current_next_node] + self.momentum * self.delta_weights[current_node][current_next_node] + self.learning_rate * self.local_gradients[current_next_node] * self.outputs[current_node]
                    self.delta_weights[current_node][current_next_node] = new_weight - self.weights[current_node][current_next_node]
                    self.weights[current_node][current_next_node] = new_weight
        for i in range(1, len(self.biases)):
            new_bias = self.biases[i] + self.momentum  * self.delta_biases[i] + self.learning_rate * self.local_gradients[i]
            self.delta_biases[i] = new_bias - self.biases[i]
            self.biases[i] = new_bias
            
    # Fit
    def fit(self, data, batch_size, max_iter): # data = array of arrays
        #data[0] ke n merupakan label
        #nodes_n_in_hidden_layers[0] merupakan jumlah input
        self.nodes_n_in_hidden_layers.insert(0, len(data[0])-1) 
        
        inputs = data[0]
        self.target = inputs.pop(len(data[0])-1)
        self.inputs.append(inputs)
        n_nodes = 0
        init_weight = 1 # Weights diinisalisasi 0
        
        # Inisialisasi output, bias, local gradient, v, dan delta bias di setiap node pada layer
        for i in range(0, len(self.nodes_n_in_hidden_layers)):
            l_nodes = []
            for j in range(0, self.nodes_n_in_hidden_layers[i]):
                self.outputs.append(0)
                self.v.append(0)
                self.biases.append(0) #asumsi x bias = 1
                self.local_gradients.append(0)
                self.delta_biases.append(0)
                l_nodes.append(n_nodes)
                n_nodes += 1
            self.layer_nodes.append(l_nodes)
        
        for i in range(0, n_nodes):
            values = []
            for j in range(0, n_nodes):
                values.append(None)
            self.weights.append(values)
            self.delta_weights.append(values)
            
        current_node = 0
        for i in range(0, len(self.nodes_n_in_hidden_layers)-1):
            if (i < len(self.nodes_n_in_hidden_layers)-1):
                next_layer_first_node = current_node + self.nodes_n_in_hidden_layers[i]
                for j in range(0, self.nodes_n_in_hidden_layers[i]):
                    for k in range(0, self.nodes_n_in_hidden_layers[i+1]):
                        self.weights[current_node][k+next_layer_first_node] = init_weight
                        self.local_gradients[current_node] = 0
                        self.delta_weights[current_node][k+next_layer_first_node] = init_weight
                    current_node += 1
        
        n_batch = math.ceil(len(data)/batch_size)
        
        n_iter = 0
        while (n_iter < max_iter):
            
            datum_idx = 0
            for i in range(0, n_batch):
                j = 0
                while (j < batch_size):
                    if (datum_idx < len(data)):
                        self.feed_forward(datum_idx)
                        self.back_propagation()
                        datum_idx += 1
                        j += 1
                    else:
                        j = batch_size + 1
                self.update_weight()
                # Mengembalikan local_gradient menjadi 0
                current_node = 0
                for i in range(0, len(self.nodes_n_in_hidden_layers)-1):
                    if (i < len(self.nodes_n_in_hidden_layers)-1):
                        next_layer_first_node = current_node + self.nodes_n_in_hidden_layers[i]
                        for j in range(0, self.nodes_n_in_hidden_layers[i]):
                            for k in range(0, self.nodes_n_in_hidden_layers[i+1]):
                                self.local_gradients[current_node] = 0
                            current_node += 1
                        
            n_iter += 1
            
    
    # Predict
    def predict(self, data_test):
        test_outputs = []
        feed_forward_result = []
        
        for i in range (0, len(data_test)):
            feed_forward_result = self.feed_forward(i)
            tes_outputs.append(feed_forward_result[-1])
        
        return test_outputs

## Fitting dengan Hasil Implementasi Kelompok

In [87]:
nn = NeuralNet([2], 0.25, 0.0001) 
nn.fit([[0.1,0.9,0.9]], 1, 1)

## Implementasi dengan Keras

In [116]:
from keras.models import Sequential
from keras.layers import Dense

class KerasNeuralNet:
    
    # Construction
    def __init__(self, nodes_n_in_hidden_layers, learning_rate, momentum):
        nodes_n_in_hidden_layers.append(1) # satu node buat output
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.model = Sequential()
        self.model.add(Dense(output_dim=nodes_n_in_hidden_layers[0], input_dim=4, activation="sigmoid"))
        for i in range(1, len(nodes_n_in_hidden_layers)):
            self.model.add(Dense(output_dim=nodes_n_in_hidden_layers[i], input_dim=nodes_n_in_hidden_layers[i-1], activation='sigmoid'))
        sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        self.model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
        
    def fit(self, X, Y, batch_size):
        print(X, Y, batch_size)
        self.model.fit(X, Y, batch_size=batch_size)
    
    def predict(self, X):
        return self.model.predict(X)

## Fitting dengan Keras

In [118]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

# Load data
dataframe = pd.read_csv('tennis.csv')

# Transform outlook, temperature, humidity, and windy to numerical values
le = preprocessing.LabelEncoder()
encoded = dataframe.apply(le.fit_transform)
dataset = encoded.values

# X and Y values
X = dataset[:,0:4]
Y = dataset[:,4]

# Rescale min and max for X
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X)

knn = KerasNeuralNet([12,8,1], 0.25, 0.0001)

# Train model
knn.fit(rescaledX, Y, 1)

# Predict
knn.predict(rescaledX)

/home/suzaneringoringo/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/suzaneringoringo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=4, activation="sigmoid", units=12)`
  if sys.path[0] == '':
/home/suzaneringoringo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=12, activation="sigmoid", units=8)`
  
/home/suzaneringoringo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=8, activation="sigmoid", units=1)`
  
/home/suzaneringoringo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `

[[ 1.   0.5  0.   0. ]
 [ 1.   0.5  0.   1. ]
 [ 0.   0.5  0.   0. ]
 [ 0.5  1.   0.   0. ]
 [ 0.5  0.   1.   0. ]
 [ 0.5  0.   1.   1. ]
 [ 0.   0.   1.   1. ]
 [ 1.   1.   0.   0. ]
 [ 1.   0.   1.   0. ]
 [ 0.5  1.   1.   0. ]
 [ 1.   1.   1.   1. ]
 [ 0.   1.   0.   1. ]
 [ 0.   0.5  1.   0. ]
 [ 0.5  1.   0.   1. ]] [0 0 1 1 1 0 1 0 1 1 1 1 1 0] 1
Epoch 1/1
14/14 [==============================] - 1s 80ms/step - loss: 0.6516 - acc: 0.6429


array([[ 0.62409371],
       [ 0.62643188],
       [ 0.62993127],
       [ 0.62661719],
       [ 0.62857229],
       [ 0.63095552],
       [ 0.63384718],
       [ 0.62387693],
       [ 0.62565267],
       [ 0.62803423],
       [ 0.62775546],
       [ 0.63179791],
       [ 0.63118571],
       [ 0.62894696]], dtype=float32)